### Q1: Running Elastic
Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

##### terminal
@sonalishintre ➜ /workspaces/Large-Language-Model--Zoomcamp (main) $ curl http://localhost:9200

{
  "name" : "c8133064d7e3",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "bJUBWiJIQ3CvaKPfEXqbhQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}

#### Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
#documents_raw[1]

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Q3: Searching 

In [9]:
query = {
    "query": {
        "multi_match": {
            "query": "How do execute a command on a Kubernetes pod?",
            "fields": ["question^4", "text"],  # Boost question field
            "type": "best_fields"
        }
    }
}

In [19]:
for i, doc in enumerate(documents):
    res = requests.post(f"http://localhost:9200/llm_docs/_doc/{i}", json=doc)
    #print(res.status_code, res.json())

In [26]:
response = requests.get("http://localhost:9200/llm_docs/_search", json=query)
results = response.json()
results.keys()

dict_keys(['took', 'timed_out', '_shards', 'hits'])

In [30]:
results.values()

dict_values([24, False, {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.55304, 'hits': [{'_index': 'llm_docs', '_id': '604', '_score': 44.55304, '_ignored': ['text.keyword'], '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'llm_docs', '_id': '747', '_score': 36.05889, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', 'course': 'machine-learning-zoomcamp'}}, {'_index'

In [31]:
print("score:", results["hits"]["hits"][0]["_score"])
print("result:", results["hits"]["hits"][0]["_source"])

score: 44.55304
result: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}


#### Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

Return 3 results. What's the 3rd question returned by the search engine?

How do I debug a docker container?
How do I copy files from a different folder into docker container’s working directory?
How do Lambda container images work?
How can I annotate a graph?

In [46]:
query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    },
    "size": 3
}

In [47]:
response = requests.get("http://localhost:9200/llm_docs/_search", json=query)
results = response.json()

In [48]:
# top 3 questions
print("Top 3 Results:")
for i, hit in enumerate(results["hits"]["hits"]):
    print(f"{i+1}. {hit['_source']['question']}")

Top 3 Results:
1. How do I debug a docker container?
2. How do I copy files from my local machine to docker container?
3. How do I copy files from a different folder into docker container’s working directory?


In [49]:
## Delete the index if it already exists (optional, for clean start)
# requests.delete("http://localhost:9200/llm_docs")

# # Create the index with proper mapping
# mapping = {
#     "mappings": {
#         "properties": {
#             "course": {"type": "keyword"},
#             "question": {"type": "text"},
#             "text": {"type": "text"}
#         }
#     }
# }

# response = requests.put("http://localhost:9200/llm_docs", json=mapping)

# print("Index created:", response.status_code, response.json())

# for i, doc in enumerate(documents):
#     res = requests.post(f"http://localhost:9200/llm_docs/_doc/{i}", json=doc)
#     if res.status_code not in (200, 201):
#         print(f"Failed to index document {i}: {res.status_code}, {res.text}")

# res = requests.get("http://localhost:9200/llm_docs/_count")
# print("Total documents:", res.json()["count"])

#### Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

What's the length of the resulting prompt? (use the len function)

946
1446
1946
2446

In [50]:
import requests

# Define the search query
query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    },
    "size": 3
}

# Execute the search
response = requests.get("http://localhost:9200/llm_docs/_search", json=query)
results = response.json()["hits"]["hits"]

# Build context from search results
context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join(
    context_template.format(
        question=hit["_source"]["question"],
        text=hit["_source"]["text"]
    )
    for hit in results
)

# Final prompt template
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

# Build the final prompt
question = "How do copy a file to a Docker container?"
final_prompt = prompt_template.format(question=question, context=context)

# Print the final prompt
print("=== Final Prompt ===\n")
print(final_prompt)


=== Final Prompt ===

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory contain

In [ ]:
print("Prompt length:", len(final_prompt))

Prompt length: 1446


##### Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

In [59]:
!pip install tiktoken
!python3 -m pip install --upgrade pip
#encoding = tiktoken.encoding_for_model("gpt-4o")

In [58]:
import tiktoken

# Assume final_prompt is already created
encoding = tiktoken.encoding_for_model("gpt-4o")
num_tokens = len(encoding.encode(final_prompt))

print("Number of tokens:", num_tokens)


Number of tokens: 320
